In [1]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_ollama import ChatOllama
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate

from langchain_openai import OpenAI

from dotenv import load_dotenv

In [2]:
load_dotenv()

tools = [TavilySearchResults(max_results=1)] # + load_tools(["openweathermap-api"])

In [3]:
# llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)
llm = ChatOllama(model="llama3.1", temperature=0)

In [4]:
# Get the prompt to use - you can modify this!
# prompt = hub.pull("hwchase17/openai-tools-agent")

# agent = create_openai_tools_agent(llm, tools, prompt)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use the tavily_search_results_json tool for information.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construct the Tools agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [5]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "what is LangChain?"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `tavily_search_results_json` with `{'query': 'LangChain'}`


[{'url': 'https://en.wikipedia.org/wiki/LangChain', 'content': 'In October 2023 LangChain introduced LangServe, a deployment tool designed to facilitate the transition from LCEL (LangChain Expression Language) prototypes to production-ready applications.[5]\nIntegrations[edit]\nAs of March 2023, LangChain included integrations with systems including Amazon, Google, and Microsoft Azure cloud storage; API wrappers for news, movie information, and weather; Bash for summarization, syntax and semantics checking, and execution of shell scripts; multiple web scraping subsystems and templates; few-shot learning prompt generation support; finding and summarizing "todo" tasks in code; Google Drive documents, spreadsheets, and presentations summarization, extraction, and creation; Google Search and Microsoft Bing web search; OpenAI, Anthropic, and Hugging Face language models; iFixit repair guides and wikis search and summari

{'input': 'what is LangChain?',
 'output': "LangChain is an open-source project launched in October 2022 by Harrison Chase while working at Robust Intelligence. It serves as a language model integration framework, allowing for the creation of custom AI models and applications. LangChain has integrations with various systems, including cloud storage services, API wrappers, web scraping tools, and more. It also supports the use of language models from providers like OpenAI, Anthropic, and Hugging Face.\n\nAs of April 2023, LangChain had incorporated and raised over $20 million in funding at a valuation of at least $200 million from venture firm Sequoia Capital. The project has gained popularity, with hundreds of contributors on GitHub, trending discussions on Twitter, and lively activity on the project's Discord server.\n\nLangChain is designed to facilitate the transition from prototypes to production-ready applications through its deployment tool, LangServe. It can read from more than 

# Adding in Memory

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [7]:
message_history = ChatMessageHistory()

In [8]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [9]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `tavily_search_results_json` with `{'query': 'hi! I'}`


[{'url': 'https://dictionary.cambridge.org/dictionary/english/hi', 'content': "HI | English meaning - Cambridge Dictionary Meaning of hi in English hi Hi, there! Hi, how are you doing? HI (Definition of hi from the Cambridge Advanced Learner's Dictionary & Thesaurus © Cambridge University Press) hi | American Dictionary hi Hi, how are you doing? (Definition of hi from the Cambridge Academic Content Dictionary © Cambridge University Press) What is the pronunciation of hi? Translations of hi hi More meanings of hi Free word lists and quizzes from Cambridge Word lists shared by our community of dictionary fans English–Arabic English–Bengali English–Catalan English–Czech English–Danish English–Gujarati English–Hindi English–Korean English–Malay English–Marathi English–Russian English–Tamil English–Telugu English–Thai English–Turkish English–Ukrainian English–Urdu English–Vietnamese To add hi to a word list please sign up o

{'input': "hi! I'm bob",
 'chat_history': [],
 'output': "Hi Bob! It's nice to meet you. Would you like to talk about something specific or just say hello?"}

In [10]:
agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `tavily_search_results_json` with `{'query': 'Bob'}`


[{'url': 'https://www.bankofbaroda.in/personal-banking/digital-products/instant-banking/bob-world-internet-banking', 'content': 'bob World Internet application is one of the most secure Internet Banking Application in industry as per customer survey conducted by Bank from time to time. It uses 128 bit encryption methodology. It uses strong encryption algorithm using AES 256. It performs device validation using OTP. Personal Assurance Message (PAM) to identify genuine'}]Your name is Bob!

> Finished chain.


{'input': "what's my name?",
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content="Hi Bob! It's nice to meet you. Would you like to talk about something specific or just say hello?")],
 'output': 'Your name is Bob!'}

In [11]:
weather_results = agent_with_chat_history.invoke(
    {"input": "I live in London.  What is the weather like where I live?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)['output']

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `tavily_search_results_json` with `{'query': 'London weather'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'London', 'region': 'City of London, Greater London', 'country': 'United Kingdom', 'lat': 51.5171, 'lon': -0.1062, 'tz_id': 'Europe/London', 'localtime_epoch': 1733243883, 'localtime': '2024-12-03 16:38'}, 'current': {'last_updated_epoch': 1733243400, 'last_updated': '2024-12-03 16:30', 'temp_c': 4.3, 'temp_f': 39.7, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 2.5, 'wind_kph': 4.0, 'wind_degree': 159, 'wind_dir': 'SSE', 'pressure_mb': 1023.0, 'pressure_in': 30.21, 'precip_mm': 0.05, 'precip_in': 0.0, 'humidity': 87, 'cloud': 0, 'feelslike_c': 3.8, 'feelslike_f': 38.8, 'windchill_c': 7.0, 'windchill_f': 44.5, 'heatindex_c': 7.2, 'heatindex_f': 44.9, 'dewpoint_c': 1.0, 'dewpoint_f': 33.7, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 3.4, 'gus

In [12]:
from openai import OpenAI
from IPython.display import Audio

client = OpenAI()
response = client.audio.speech.create(
    model='tts-1',
    voice='alloy',
    input=weather_results,
)

Audio(response.read())